In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime

Using TensorFlow backend.
c:\users\robotics\anaconda3\envs\faster_rcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\robotics\anaconda3\envs\faster_rcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\robotics\anaconda3\envs\faster_rcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\robotics\an

In [2]:
ratio = 8
channels = 3
HR_size = np.array([480,640])#
LR_size = HR_size // ratio
H_h = HR_size[0]
H_w = HR_size[1]
L_h = LR_size[0]
L_h = LR_size[1]
print(HR_size[0])
print(LR_size)

480
[60 80]


In [3]:
class DataLoader():
    def __init__(self):
        pass
#         self.ratio = ratio
#         self.HR_size = HR_size
#         self.LR_size = HR_size / ratio
#         self.H_h = HR_size[0]
#         self.H_w = HR_size[1]
#         self.L_h = LR_size[0]
#         self.L_h = LR_size[1]
        
    
    
    def load_data(self, batch_size = 1):
        files = glob.glob("../../images/train/*.png", recursive=True)
        batch_images = random.sample(files, batch_size)
    
        HR_images = []
        LR_images = []
    
        for img_path in batch_images:
            img = Image.open(img_path)

            HR_image = img.resize(H_w, H_h)  #(64, 64)
            HR_image = img.resize(L_w, L_h)
            HR_image = np.array(HR_image)
            #img_hr = (img_hr - 127.5) / 127.5
            HR_image = np.array(HR_image)
            #img_lr = (img_lr - 127.5) / 127.5

#             if not is_testing and np.random.random() < 0.5:
#                 img_hr = np.fliplr(img_hr)
#                 img_lr = np.fliplr(img_lr)

            HR_images.append(HR_image)
            LR_images.append(LR_image)
        
        HR_images = HR_images / 127.5 - 1
        LR_images = LR_images / 127.5 - 1
        
        return HR_images, LR_images

#------------------------------------------------------------------#
  
#------------------------------------------------------------------#

class predDataLoader():          
    def load_data(self, batch_size, counter):
        random.seed(counter)
        np.random.seed(counter)

        files = glob.glob("../../images/test/*.png", recursive=True)
        batch_images = random.sample(files, batch_size)

        imgs_or = []
        for img_path in batch_images:
            img = Image.open(img_path)

            img_or = np.array(img)
            imgs_or.append(img_or)

        imgs_or = np.array(imgs_or) / 127.5 - 1.

        return imgs_or
    
    
#------------------------------------------------------------------#
  
#------------------------------------------------------------------#

class SRGAN():
    def __init__(self):
        
        # Input shape
#         self.channels = 3
#         self.lr_height = 288                 # Low resolution height
#         self.lr_width = 384                  # Low resolution width
#         self.lr_shape = (self.lr_height, self.lr_width, self.channels)
#         resLevel = 2 #
#         self.hr_height = self.lr_height*resLevel  # High resolution height
#         self.hr_width = self.lr_width*resLevel     # High resolution width
#         self.hr_shape = (self.hr_height, self.hr_width, self.channels)

        #残差ブロックの数
        # Number of residual blocks in the generator
        self.n_residual_blocks = 16 #

        optimizer = Adam(0.0002, 0.5)

        #img_hrの特徴量をVGG19で算出する
        # We use a pre-trained VGG19 model to extract image features from the high resolution
        # and the generated high resolution images and minimize the mse between them
        self.VGG = self.build_vgg()
        self.VGG.trainable = False
        self.VGG.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        
        #データはここで読み込まれる
        # Configure data loader
        #self.dataset_name = 'img_align_celeba'
        self.data_loader = DataLoader()
        self.pred_data_loader = predDataLoader()

        #Dのサイズ
        # Calculate output shape of D (PatchGAN)
        patchH = int(H_h / 2**4) #
        patchW = int(H_w / 2**4) #
        self.disc_patch = (patchH, patchW, 1) 

        #DとGのチャンネル設定
        # Number of filters in the first layer of G and D
        self.gf = 60 #gf
        self.df = 60 #df
        #Dビルドとコンパイル
        # Build and compile the discriminator
        self.Discriminator = self.build_discriminator()
        self.Discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #Gのビルド
        # Build the generator
        self.Generator = self.build_generator()

        # High res. and low res. images
        HR_image = Input(shape = (H_h, H_w, channels))
        LR_image = Input(shape = (L_h, H_w, channels))

        #Gで生成されたhrのimg
        # Generate high res. version from low res.
        SR_image = self.Generator(LR_image)

        #hrのimgの特徴量の算出
        # Extract image features of the generated img
        SR_features = self.VGG(SR_image)

        # For the combined model we will only train the generator
        self.Discriminator.trainable = False

        #学習モデルコンパイル
        # Discriminator determines validity of generated high res. images
        validity = self.Discriminator(SR_image)

        self.combined = Model([LR_image, HR_image], [validity, SR_features])
        self.combined.compile(loss=['binary_crossentropy', 'mse'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)
        
    #------------------------------------------------------------------#
    
    def build_vgg(self):
        """
        Builds a pre-trained VGG19 model that outputs image features extracted at the
        third block of the model
        """
        VGG = VGG19(weights="imagenet")
        # Set outputs to outputs of last conv. layer in block 3
        # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
        VGG.outputs = [VGG.layers[9].output]

        img = Input(shape = (H_h, H_w, channels))

        # Extract image features
        img_features = VGG(img)

        return Model(img, img_features)
    
    #------------------------------------------------------------------#
    
    #Gの中身
    def build_generator(self):

        #残差ブロックの中身
        def residual_block(layer_input, n_filters):
            """Residual block described in paper"""
            d = Conv2D(n_filters, kernel_size=3, strides=1, padding='same')(layer_input)
            d = Activation('relu')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Conv2D(n_filters, kernel_size=3, strides=1, padding='same')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Add()([d, layer_input])
            return d

        #解像度を2倍にするUpSampling
        def deconv2d(layer_input):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input) #
            u = Conv2D(120, kernel_size=3, strides=1, padding='same')(u) #
            u = Activation('relu')(u)
            return u

        # Low resolution image input
        img_lr = Input(shape=self.lr_shape)

        # Pre-residual block
        c1 = Conv2D(60, kernel_size=9, strides=1, padding='same')(img_lr) #
        c1 = Activation('relu')(c1)

        # Propogate through residual blocks
        r = residual_block(c1, self.gf)
        for _ in range(self.n_residual_blocks - 1):
            r = residual_block(r, self.gf)

        #去の残差ブロックと組み合わせる
        # Post-residual block
        c2 = Conv2D(60, kernel_size=3, strides=1, padding='same')(r) #
        c2 = BatchNormalization(momentum=0.8)(c2)
        c2 = Add()([c2, c1])

        # Upsampling
        u2 = deconv2d(c2)
        #u2 = deconv2d(u1)

        # Generate high resolution output
        gen_hr = Conv2D(self.channels, kernel_size=9, strides=1, padding='same', activation='tanh')(u2)

        return Model(img_lr, gen_hr)
    
    
    
    

In [10]:
data = DataLoader()